# Command line tool for tracking

In [1]:
from laptrack import LapTrack
from laptrack import data_conversion
import geff
import tempfile
import napari
from pathlib import Path
import zarr
import numpy as np

tmp_path = Path(tempfile.gettempdir())
print(f"Temporary files will be stored in: {tmp_path}")
!open {tmp_path}

Temporary files will be stored in: /var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T


## Tracking CSV coordinates

In [2]:
!laptrack track --help

usage: laptrack track [--csv_path CSV_PATH]
                      [--track_geff_path TRACK_GEFF_PATH] --output_path
                      OUTPUT_PATH [--metadata_path METADATA_PATH]
                      [--frame_col FRAME_COL] --coordinate_cols
                      [COORDINATE_COLS ...] [-h] [--metric METRIC]
                      [--cutoff CUTOFF]
                      [--gap_closing_metric GAP_CLOSING_METRIC]
                      [--gap_closing_cutoff GAP_CLOSING_CUTOFF]
                      [--gap_closing_max_frame_count GAP_CLOSING_MAX_FRAME_COUNT]
                      [--splitting_metric SPLITTING_METRIC]
                      [--splitting_cutoff SPLITTING_CUTOFF]
                      [--merging_metric MERGING_METRIC]
                      [--merging_cutoff MERGING_CUTOFF]
                      [--track_start_cost TRACK_START_COST]
                      [--track_end_cost TRACK_END_COST]
                      [--segment_start_cost SEGMENT_START_COST]
                      [--

In [5]:
!cat command_line_data/metadata.json | head

{
  "geff_version": "0.5.1.dev1+ge6d3dc6",
  "directed": true,
  "axes": [
    {
      "name": "frame",
      "type": "time",
      "unit": "frame",
      "min": null,
      "max": null


In [28]:
zarr_path = tmp_path / "images.zarr"
print(zarr_path)
!rm -r {zarr_path}
!cp -r command_line_data/images.zarr {zarr_path}
!ls {zarr_path}

/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/images.zarr
image  labels  zarr.json


In [29]:
!laptrack track --csv_path command_line_data/regionprops.csv \
    --metadata_path command_line_data/metadata.json \
    --output_path {zarr_path}/tracks.geff \
    --coordinate_cols y x \
    --frame_col frame \
    --metric sqeuclidean \
    --cutoff 225 \
    --splitting_metric sqeuclidean \
    --splitting_cutoff 225 \

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:746: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  return cls.__pydantic_validator__.validate_json(
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:746: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter', 'picometer', 'terameter', 'yard', 'y

## Tracking GEFF to find missing edges

Making a geff file with missing edges

In [30]:
geff_tree, meta = geff.read_nx(zarr_path / "tracks.geff")
all_edges = list(geff_tree.edges)
for e in np.random.choice(len(geff_tree.edges), 20, replace=False):
    e2 = all_edges[e]
    geff_tree.remove_edge(e2[0], e2[1])
!rm -r {zarr_path}/tracks_missing_edges.geff
geff.write_nx(
    geff_tree,
    zarr_path / "tracks_missing_edges.geff",
    metadata=meta,
)

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  v_list = validator(input_value)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter', 'picometer', 'teramet

rm: cannot remove '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/images.zarr/tracks_missing_edges.geff': No such file or directory


/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/main.py:253: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter'

In [13]:
!rm -r {zarr_path}/tracks_retracked.geff
!laptrack track --track_geff_path {zarr_path}/tracks_missing_edges.geff \
    --output_path {zarr_path}/tracks_retracked.geff \
    --coordinate_cols y x \
    --frame_col frame \
    --metric sqeuclidean \
    --cutoff 225 \
    --splitting_metric sqeuclidean \
    --splitting_cutoff 225 \

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  v_list = validator(input_value)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter', 'picometer', 'teramet

## Tracking images by overlap

In [16]:
!laptrack overlap_track --help

usage: laptrack overlap_track --labels_path LABELS_PATH
                              [--image_path IMAGE_PATH]
                              [--metadata_path METADATA_PATH] --output_path
                              OUTPUT_PATH [--append_relative_object] [-h]
                              [--metric METRIC] [--cutoff CUTOFF]
                              [--gap_closing_metric GAP_CLOSING_METRIC]
                              [--gap_closing_cutoff GAP_CLOSING_CUTOFF]
                              [--gap_closing_max_frame_count GAP_CLOSING_MAX_FRAME_COUNT]
                              [--splitting_metric SPLITTING_METRIC]
                              [--splitting_cutoff SPLITTING_CUTOFF]
                              [--merging_metric MERGING_METRIC]
                              [--merging_cutoff MERGING_CUTOFF]
                              [--track_start_cost TRACK_START_COST]
                              [--track_end_cost TRACK_END_COST]
                              [--segment_s

In [33]:
!ls {zarr_path}/labels/c

0  1


In [46]:
!laptrack overlap_track --labels_path {zarr_path}/labels \
    --image_path {zarr_path}/image \
    --output_path {zarr_path}/tracks_overlap.geff \
    --append_relative_objects 

Traceback (most recent call last):
  File "/Users/fukai/projects/laptrack/.venv/bin/laptrack", line 6, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/fukai/projects/laptrack/src/laptrack/__main__.py", line 233, in main
    overlap_track(args)
  File "/Users/fukai/projects/laptrack/src/laptrack/__main__.py", line 197, in overlap_track
    related_obj_path = args.labels_path.absolute().relative_to(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/fukai/.local/share/uv/python/cpython-3.12.11-macos-x86_64-none/lib/python3.12/pathlib.py", line 682, in relative_to
    raise ValueError(f"{str(self)!r} is not in the subpath of {str(other)!r}")
ValueError: '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/images.zarr/labels' is not in the subpath of '/var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T/images.zarr/tracks_overlap.geff'
